In [4]:
import pandas as pd
import numpy as np
import os
import datetime
import warnings

warnings.filterwarnings("ignore")

In [2]:
DATA_PATH = "../data/"

In [5]:
# submission from
submission = pd.read_csv(f'{DATA_PATH}godaddy-submissions/submission_1.3730.csv')
submission['cfips'] = submission['row_id'].map(lambda x: int(x.split('_')[0]))

# get adult_population
column_names = ['GEO_ID','NAME','S0101_C01_026E']
df2021 = pd.read_csv(f'{DATA_PATH}census-data-for-godaddy/ACSST5Y2021.S0101-Data.csv', usecols=column_names)
df2021 = df2021.iloc[1:]
df2021['S0101_C01_026E'] = df2021['S0101_C01_026E'].astype('int')
df2021['cfips'] = df2021.GEO_ID.apply(lambda x: int(x.split('US')[-1]))
adult2021 = df2021.set_index('cfips').S0101_C01_026E.to_dict()

submission['adult2021'] = submission['cfips'].map(adult2021)

# rounding to the nearest integer
submission['microbusiness_density'] = (
    np.round(submission['microbusiness_density'] * submission['adult2021'] / 100) / submission['adult2021'] * 100
)

# submit
currentDate = "".join(str(datetime.datetime.now())[:10].split("-"))
nextVersion = 1
try:
    file_list = os.listdir(f'submission/')
    if file_list:
        maxVersion = 0
        for _file in file_list:
            if f'submission_{currentDate}_' in _file:
                maxVersion = max(maxVersion, int(_file.split("_")[-1][:-4]))
        nextVersion = maxVersion + 1
    submission[['row_id', 'microbusiness_density']].to_csv(f'submission/submission_{currentDate}_{nextVersion}.csv', index=False)
except:
    pass